Zadanie 1.

Zaprojektuj klasę dla $kd$-drzewa.

In [40]:
import math
# przy wykonaniu tego zadania korzystam z biblioteki pandas (również, ponieważ trochę ją poznałem),
# ponieważ dostarcza pewne ułatwiające funkcjonalności, jak:
# DataFrames - bardzo wygodny pojemnik na dane, funkcja '.iloc' i '.sort_values'.
import time

import pandas as pd
from binarytree import tree, Node

class kd_Tree:

    def __init__(self, data):
        self.data = data
        self.tree = None

    def _build(self, points, depth):

        # wyliczamy wymiar k (zakładamy, że dane otrzymujemy jako dataframe) - pandas
        k = len(points.columns)

        # ustalamy wybór kolumny (ma będą wybierane po kolei z indeksami [0], [1], [0] , [1] itd.)
        _axis = depth % k

        # wybieramy kolumnę
        _column = points.columns[_axis]
        if len(points) == 0:
            return None

        # sortyjemy DataFrame rosnąco, aby obliczyć medianę
        objects_list = points.sort_values(by= [_column], ascending= True)

        # obliczamy medianę
        median_idx = int((len(objects_list)//2))

        node = Node(int(round(objects_list.iloc[median_idx][_column],3)))
        node.left = self._build(objects_list.iloc[0:median_idx], depth+1)
        node.right = self._build(objects_list.iloc[median_idx+1:], depth+1)
        return node

    def build(self):
        self.tree = self._build(self.data, depth= 0)

test_df = pd.DataFrame(data= [[2,3],[5,4],[9,6],[4,7],[8,1],[7,2],[6,1],[10,2],[11,10],[12,5]], columns= ['X', 'Y'])

kd = kd_Tree(test_df)
kd.build()
print(kd.tree)

# Wybrałem sposób tworzenia drzewa przy użyciu Node (binarytree), więc w węzłach muszą znajdywać się pojedyńcze liczby,
# Zbiór danych, jednak pozwala jednoznacznie stwierdzić jak wygląda podział, drzewo można by było równoważnie zapisać z
# tuple'ami w węzłach, lecz sposób podany przeze mnie, jest również dopuszczony. Trzeba pamiętać, że (od góry) wartości
# występują w kolejności: wartość z kolumny X, Y, X, Y ... Więc na samej górze jest [8,1] , później [2,3] i [9,6] itd.


      ____8_____
     /          \
    3__         _6
   /   \       /  \
  7     5     12   11
 /     /     /
1     7     2



Zadanie 2.

Zaimplementuj wzorzec projektowy łańcuch odpowiedzialności na przykładzie obsługi żądania _http_ (symulacja),
w którym przed faktycznym kodem obsługi błędu ma zostać sprawdzone:
- czy użytkownik może wysłać danego typu żądania
- czy żądanie nie dotyczy pliku
- czy liczba żądań na minutę nie jest przekroczona
- czy liczba żądań na minutę nie jest przekroczona dla zalogowanego użytkownika
- czy przesłany formularz nie jest próbą `sql incjection`

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any, Optional
from datetime import datetime
import re
import time

# Tworzę zmienną globalną role, która będzie potrzebna w handlerach do podejmowania decyzji
role = None

#Klasa, której powołane obiekty będą symulować request http
class httpSimRequest:
    def __init__(self, Type: str, File: Any, Role: str, username: str, password: str):
        self.httpReq = {
            'URL': 'https://someWebsite.com',
            'HTTP_ACCEPT': '*/*',
            'Type': Type,
            'File': File,
            'HTTP_HOST': 'MyHost',
            'HTTP_CONNECTION': 'keep-alive',
            'Minute': datetime.now(),

            # Czy użytkownik jest zalogowany sprawdzam w Role -> gdy jest zalogowany to rola to LoogedUser, jest to
            # potrzebna informacja do m.in. sprawdzenia czy liczba żądań na minutę dla zalogowanego użytkownika nie jst za duża

            'Role': Role,
            'Form': {
                'Username': username,
                'Password': password
            },
            'HTTP_ACCEPT_ENCODING': ['gzip', 'deflate']
        }
        global role
        role = self.httpReq['Role']

class Handler(ABC):

    @abstractmethod
    def set_next(self, handler: Handler) -> Handler:
        pass

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        pass

class AbstractHandler(Handler):

    _next_handler: Handler = None

    def set_next(self, handler: Handler) -> Handler:
        self._next_handler = handler
        return handler

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)

        return ''

class TypeHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        # Zakładamy, że użytkownik 'user' może wysłać tylko request get, loggedUser - get, head i put a admin wszystkie
        types = ['get','head','put','post','delete','patch']
        roles = ['admin','loggedUser','user']
        if request in types:
            if role == roles[0]:
                return f"Użytkownik może wysłać żądanie typu: {request}"

            elif role == roles[1]:
                if types.index(request) < 3:
                    return f"Użytkownik może wysłać żądanie typu: {request}"
                else:
                    raise Exception(f"Użytkownik nie może wysłać żądania typu: {request}")

            else:
                if types.index(request) == 0:
                    return f"Użytkownik może wysłać żądanie typu: {request}"
                else:
                    raise Exception(f"Użytkownik nie może wysłać żądania typu: {request}")

        else:
            return super().handle(request)


class FileHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == 'file':
            raise Exception("Użytkownik nie może wysłać żądania! Żądanie dotyczy pliku.")
        else:
            return super().handle(request)

# Potrzebna zmienna globalna do przechowywania wartości do przyszłych requestów
RPMCounter = {'min': -1, 'numOfReq': 0}

# Dodać exceptions, gdy nie możemy wysłać żądania
class RequestsPerMinuteHandler(AbstractHandler):
    global RPMCounter
    def handle(self, request: Any) -> str:
        roles = ['admin','loggedUser','user']

        # Zakładam, że limit żądań na minutę admin-7 loggedUser-5 user-3
        # przyjmuje tak niskie liczby dla łatwiejszego sprawdzania
        if isinstance(request, datetime):
            if RPMCounter['min'] == -1:
                RPMCounter['min'] = int(request.strftime('%M'))
                RPMCounter['numOfReq'] = 1
            elif RPMCounter['min'] == int(request.strftime('%M')):
                RPMCounter['numOfReq'] += 1
            else:
                RPMCounter['min'] = int(request.strftime('%M'))
                RPMCounter['numOfReq'] = 1
            if role == roles[0]:
                if RPMCounter['numOfReq'] < 8:
                    return f"Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę."
                else:
                    raise Exception("Użytkownik nie może wysłać żądania! Użytkownik przekroczył maksymalną liczbę żądań na minutę.")

            elif role == roles[1]:
                if RPMCounter['numOfReq'] < 6:
                    return f"Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę."
                else:
                    raise Exception("Użytkownik nie może wysłać żądania! Użytkownik przekroczył maksymalną liczbę żądań na minutę.")
            else:
                if RPMCounter['numOfReq'] < 4:
                    return f"Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę."
                else:
                    raise Exception("Użytkownik nie może wysłać żądania! Użytkownik przekroczył maksymalną liczbę żądań na minutę.")
        else:
            return super().handle(request)

# Dla ułatwienia rozwiązania zakładam, że użytkownik może podać SQL injection tylko w
# Parameters (domyślnie np. ustaleniem URL itd. zajmuje się np. strona). W tym
# rozwiązaniu w haśle nie wolno używać znaków specjalnych i spacji. Aby dokładniej wykryć SQL
# injection można posłużyć się jednym z wielu narzędzi dostępnych online jak np. py-find-injection 0.1.1

class SqlInjectionHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if type(request) == dict:
            for param in request:
                regex = re.compile("\'|\"|\(|\[|\*|=| |\\|/|;|-", re.IGNORECASE)
                if regex.search(request[param]):
                    raise Exception("Użytkownik nie może wysłać żądania! Użytkownik użył niedozwolonych znaków: możliwe sql injection.")


            return 'W parametrze nie występuje sql injection.'

        else:
            return super().handle(request)


def client_code(handler: Handler, data) -> None:

    # W przypadku TypeHandler i RequestPerMinuteHandler, upewniam się, że te klasy znają
    # wartość pola role z żądania http (będzie w nich potrzebna).
    notHandled = []
    try:
        for i in data:

            result = handler.handle(data[i])

            if result:
                print(f"{result}", end='\n\n')
            else:
                notHandled.append(i)
        print('Nieobsłużone pola: ')
        for i in notHandled:
            print(i)

    except Exception as e:
        print(str(e).upper())

# Aby zasymulować otwieranie pliku string (nie jest potrzebny dodatkowy plik do testu)
# Tak na prawdę, w tym przypadku nie ma to znaczenia, ponieważ musimy sprawdzić, czy operacja NIE odnosi się do pliku,
# więc będziemy tylko sprawdzać czy ma jakiekolwiek pliki w requescie (has file?) -> czy w miejscu na plik jest cokolwiek

# wywołanie:
Type = TypeHandler()
File = FileHandler()
RPM = RequestsPerMinuteHandler()
SQL = SqlInjectionHandler()

# Ustalenie kolejności wykonywania handlerów (Można wybierać tzw.
# subchain'y wpisując jako parametr w client code na 1 miejscu coś innego niż type np. File, wtedy
# subchain będzie wyglądać: File > RPM > SQL (bez type).
Type.set_next(File).set_next(RPM).set_next(SQL)

reqType = 'get'
file = None
currRole = 'user'
username = 'John123'
password = '"; select true; --"' # Próba zastosowania sql injection -> program nie może jej dopuścić

print("Chain: Type > File > RPM > SQL\n")

# for i in range(5):
request1 = httpSimRequest(reqType, file, currRole, username, password).httpReq
client_code(Type, request1)

# time.sleep(60)
# for i in range(5):
#     request2 = httpSimRequest(reqType, file, currRole, username, password).httpReq
#     client_code(Type, request2)

Chain: Type > File > RPM > SQL

Użytkownik może wysłać żądanie typu: get

Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę.

UŻYTKOWNIK NIE MOŻE WYSŁAĆ ŻĄDANIA! UŻYTKOWNIK UŻYŁ NIEDOZWOLONYCH ZNAKÓW: MOŻLIWE SQL INJECTION.


Zadanie 3.

Za pomocą dowolnego wzorca projektowego zaimplementuj mechanizm sprawdzający poprawność wyrażenia postaci:
* a + b = c (poprawne),
* (a + b = c (niepoprawne),
* a + = c (niepoprawne).